In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support import expected_conditions as EC
import urllib.request
from seleniumbase import Driver
from urllib import request
from PIL import Image
import pytesseract
from io import BytesIO

import os
from dotenv import load_dotenv

In [3]:
sites = {'인팤':['https://tickets.interpark.com/contents/search?keyword=','&rows=20&start=0&status=OPENED&status=CLOSED&status=SCHEDULED'],
        '예사':'http://ticket.yes24.com/Search/',
       '멜론':'https://ticket.melon.com/search/index.htm?q=',
       '티링':'https://www.ticketlink.co.kr/search?query='}

In [4]:
concert = pd.read_csv('data/artists_concert.csv', encoding='UTF-8')
concert

,공연명,시작일,종료일,공연시설,공연장,객석수,주최
0,"tripleS FAN-CON, Girls Never Stop",2024.07.06,2024.07.07,이화여자대학교 삼성홀,이화여자대학교 삼성홀,708,(주) 모드하우스
1,"ARTMS World Tour, Moonshot [서울]",2024.07.20,2024.07.21,이화여자대학교 삼성홀,이화여자대학교 삼성홀,708,(주) 모드하우스
2,"DAY6 3RD WORLD TOUR, FOREVER YOUNG [인천]",2024.09.20,2024.09.22,인스파이어리조트,아레나,"15,000",(주)제이와이피엔터테인먼트(JYP Entertainment)
3,온앤오프 (ONF) CONCERT: SPOTLIGHT: THE HIDDEN SHADOW,2024.09.21,2024.09.22,장충체육관,장충체육관,"4,507",WM엔터테인먼트(더블유엠엔터테인먼트)
4,"SUPER JUNIOR, D & E WORLD TOUR: ECLIPSE",2024.09.28,2024.09.29,올림픽공원,SK핸드볼경기장(펜싱경기장),"5,003",오드엔터테인먼트
...,...,...,...,...,...,...,...
101,"Stray Kids, 5-STAR Dome Tour Seoul Special(UNV...",2023.10.21,2023.10.22,고척스카이돔,고척스카이돔,"16,744",(주)제이와이피엔터테인먼트(JYP Entertainment)
102,"VANNER FANCON TOUR, VVS ADVENTURE",2023.10.14,2023.10.15,연세대학교 백주년기념관,콘서트홀,829,주식회사 클렙엔터테인먼트
103,권은비 세 번째 콘서트: QUEEN,2023.10.07,2023.10.08,블루스퀘어,Mastercard Hall(구.아이마켓홀),"1,373",울림엔터테인먼트
104,"NMIXX THE 1ST FAN CONCERT, NMIXX CHANGE UP: MI...",2023.10.07,2023.10.08,장충체육관,장충체육관,"4,507",(주)제이와이피엔터테인먼트(JYP Entertainment)


In [5]:
concert.dtypes

공연명     object
시작일     object
종료일     object
공연시설    object
공연장     object
객석수     object
주최      object
dtype: object

In [6]:
keywords = concert['공연명'].to_list()
keywords

['tripleS FAN-CON, Girls Never Stop',
 'ARTMS World Tour, Moonshot [서울]',
 'DAY6 3RD WORLD TOUR, FOREVER YOUNG [인천]',
 '온앤오프 (ONF) CONCERT: SPOTLIGHT: THE HIDDEN SHADOW',
 'SUPER JUNIOR, D & E WORLD TOUR: ECLIPSE',
 'Xdinary Heroes Concert, Closed ♭eta: v6.4',
 'IU HEREH WORLD TOUR CONCERT: THE WINNING [서울 (앵콜) ]',
 'CRAVITY FAN CON, BEYOND YOUR MEMORIES',
 'ZEROBASEONE THE FIRST TOUR, TIMELESS WORLD IN SEOUL',
 'ONEWE GOLDEN FESTA: ONEderland',
 'LUCY 1st WORLD TOUR written by FLOWER [서울 (앵콜) ]',
 'Stray Kids World Tour: dominATE [서울]',
 '어썸스테이지, Xdinary Heroes & Dragon Pony & 정수민 [부산]',
 '어썸스테이지, 임현식: BTOB [부산]',
 'BABYMONSTER PRESENTS: SEE YOU THERE [서울]',
 'Xdinary Heroes Concert: Closed beta: v6.3',
 'Red Velvet FANCON TOUR, HAPPINESS: My Dear. ReVe1uv [서울]',
 'Beyond LIVE, THE BOYZ WORLD TOUR: ZENERATION Ⅱ',
 'RIIZE FAN-CON: RIIZING DAY FINALE [서울]',
 'VANNER 1ST CONCERT: THE FLAG: A TO V [서울 (앵콜) ]',
 'DKB WORLD TOUR, DARK STRANGE [서울]',
 'TREASURE: REBOOT, FINAL [서울]',
 '어썸스테이지

In [105]:
concert_info = pd.DataFrame({'공연명': [],
                                   '좌석' : [],
                                   '가격' : [],
                                   '양도 가능' : []})

In [9]:
keywords_list = [re.split(r':|,', keyword) for keyword in keywords]
keywords_list = [[re.sub(r'\[[^\]]*\]','', s) for s in k] for k in keywords_list]
keywords_list = [[s.strip() for s in k] for k in keywords_list]

keywords_list

[['tripleS FAN-CON', 'Girls Never Stop'],
 ['ARTMS World Tour', 'Moonshot'],
 ['DAY6 3RD WORLD TOUR', 'FOREVER YOUNG'],
 ['온앤오프 (ONF) CONCERT', 'SPOTLIGHT', 'THE HIDDEN SHADOW'],
 ['SUPER JUNIOR', 'D & E WORLD TOUR', 'ECLIPSE'],
 ['Xdinary Heroes Concert', 'Closed ♭eta', 'v6.4'],
 ['IU HEREH WORLD TOUR CONCERT', 'THE WINNING'],
 ['CRAVITY FAN CON', 'BEYOND YOUR MEMORIES'],
 ['ZEROBASEONE THE FIRST TOUR', 'TIMELESS WORLD IN SEOUL'],
 ['ONEWE GOLDEN FESTA', 'ONEderland'],
 ['LUCY 1st WORLD TOUR written by FLOWER'],
 ['Stray Kids World Tour', 'dominATE'],
 ['어썸스테이지', 'Xdinary Heroes & Dragon Pony & 정수민'],
 ['어썸스테이지', '임현식', 'BTOB'],
 ['BABYMONSTER PRESENTS', 'SEE YOU THERE'],
 ['Xdinary Heroes Concert', 'Closed beta', 'v6.3'],
 ['Red Velvet FANCON TOUR', 'HAPPINESS', 'My Dear. ReVe1uv'],
 ['Beyond LIVE', 'THE BOYZ WORLD TOUR', 'ZENERATION Ⅱ'],
 ['RIIZE FAN-CON', 'RIIZING DAY FINALE'],
 ['VANNER 1ST CONCERT', 'THE FLAG', 'A TO V'],
 ['DKB WORLD TOUR', 'DARK STRANGE'],
 ['TREASURE', 'REBOOT

In [ ]:
# :이랑 , 기준으로 split
# trim()
# 리스트 맨앞(공연 가수 및 종류)으로 
# 사이트 다 돌면서 검색
# len(검색결과) != 0 이면
# 결과의 공연 기간 확인 -> 23.10 ~ 24.09 내면 가져오기
# 리스트 나머지 글자들 가져온 공연명에 다 들어가 있는지 확인
# 다 들어가있는 거만 click()
# 공연가격 가져오기 - 좌석별 차등이면 df 행 3개로 추가
# 상세정보 img OCR -> “양도|제한|예매 취소” 포함하는지 확인
# 있으면 양도 가능에 X, 없으면 O

In [11]:
caution = ['양도', '제한', '예매 취소', '예매취소', '불법']

In [104]:
def interpark_crawling(site, search_word):
    url = ''.join([sites[site][0], search_word, sites[site][1]])

    driver.get(url)
    driver.switch_to.window(driver.window_handles[0])

    concert_list = driver.find_elements(By.XPATH, '/html/body/div[1]/div/main/div/div/div[1]/div[2]/a')
    print(len(concert_list))
    if len(concert_list) == 0:
        return
    for i in range(len(concert_list)):
        period = driver.find_element(By.XPATH, f'/html/body/div[1]/div/main/div/div/div/div[2]/a[{i+1}]/ul/li[3]').text
        date = dt.datetime.strptime(period.split(' ')[0].replace('.', '-'), '%Y-%m-%d')

        if date <= to_date and date >= from_date:
            detail = driver.find_element(By.XPATH, f'/html/body/div[1]/div/main/div/div/div/div[2]/a[{i+1}]')
            detail.click()

            print(driver.window_handles)


            # 예매정보 창
            driver.switch_to.window(driver.window_handles[1])

            try:
                popup = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[3]/button')
                popup.click()
            except:
                print('nothing')
                pass
                
            driver.implicitly_wait(10)
            title = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[1]/div/div[1]/h2').text
            print(title)
            img = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div[5]/div/p/strong/img')
            image_url = img.get_attribute('src')
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
            is_caution = False
            try:
                res = request.urlopen(image_url).read()
                img = Image.open(BytesIO(res))
                text = pytesseract.image_to_string(img, lang='kor')
                is_caution = any(c in text for c in caution)
                print(is_caution)
            except:
                continue

            # 오픈 공지
            driver.get(f"https://ticket.interpark.com/webzine/paper/TPNoticeView.asp?bbsno=34&pageno=1&stext={title}&no=50434&groupno=50434&seq=0&KindOfGoods=TICKET&Genre=&sort=WriteDate")
            
            try:
                alert = Alert(driver)
                alert.accept()
            except:
                ticket = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[2]/div[2]/div/p[1]').text
                print(ticket)
                ticket = ticket.split('\n')
                price = re.split('\/|:', ticket[2])[1:]
                price = [s.strip() for s in price]
                price = [re.sub(r'[^가-힣a-zA-Z0-9 ]|^[0-9]층|원$','', s) for s in price]
                price = [s.strip() for s in price]
                price = [[s[:s.rfind(' ')], s[s.rfind(' ')+1:]] for s in price]
                for j in range(len(price)):
                    concert_info.loc[len(concert_info)] = [title, price[j][0], price[j][1], is_caution]
                    
            else:
                concert_info.loc[len(concert_info)] = [title, np.nan, np.nan, is_caution]

            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

In [106]:
concert_info

,공연명,좌석,가격,양도 가능


In [ ]:
options = Options()
service = Service(excutable_path=ChromeDriverManager().install())
driver = Driver(browser='chrome', uc=True)

from_date = dt.datetime.strptime('2023-10-01', '%Y-%m-%d')
to_date = dt.datetime.strptime('2024-09-30', '%Y-%m-%d')

for k in range(3,len(keywords_list)):
    # 검색어
    search_word = keywords_list[k][0] if len(keywords_list[k][0]) > 3 else keywords_list[k][1]
    # 크롤링 준비
    driver.implicitly_wait(10)
    driver.set_window_size(1100, 1100)

    
    for site in sites:
        # 인터파크 서치
        if site == '인팤':
            interpark_crawling(site, search_word)
            
            

3
['0EC7BD9AE6F460E03B3D17595D2C2795', 'B0B6D788E3E8E1B6C767CC4FF9FE3BB7']


In [92]:

driver = Driver(browser='chrome', uc=True)
driver.maximize_window()
driver.implicitly_wait(10)

driver.get('https://tickets.interpark.com/goods/24010841')
try:
    l = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/div[3]/button')
    l.click()
except:
    pass
print()
print(driver.window_handles)
size = driver.get_window_size()
width = size.get("width")
height = size.get("height")


print(str(width)+"px"+" "+str(height)+"px")


['B3BB76527C430BEB783E5890B04C4553']
1936px 1048px
